In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('./example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [4]:
df = pd.read_clipboard()

In [6]:
inventory = df[["inventory_id","film_id","store_id","last_update"]]
film = df[["film_id","title","description","release_year","language_id","original_language_id","rental_duration","rental_rate","length","replacement_cost","rating","last_update","special_features"]]
rental = df[["rental_id","rental_date","inventory_id","customer_id","return_date","staff_id","last_update"]]

inventory.to_sql('inventory', conn, if_exists='append', index=False)
film.to_sql('film', conn, if_exists='append', index=False)
rental.to_sql('rental', conn, if_exists='append', index=False)  


100

In [7]:
rsq("""SELECT f.title, f.rental_duration,
       -- Calculate the number of days rented
       r.return_date - r.rental_date AS days_rented
FROM film AS f
     INNER JOIN inventory AS i ON f.film_id = i.film_id
     INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

,title,rental_duration,days_rented
0,ACE GOLDFINGER,3,0.0
1,ACE GOLDFINGER,3,NaN
2,ACE GOLDFINGER,3,0.0
3,ACE GOLDFINGER,3,NaN
4,ACE GOLDFINGER,3,0.0
...,...,...,...
5839,AIRPORT POLLOCK,6,0.0
5840,AIRPORT POLLOCK,6,0.0
5841,AIRPORT POLLOCK,6,0.0
5842,AIRPORT POLLOCK,6,0.0


In [ ]:
rsq("""SELECT f.title, f.rental_duration, 
    -- Calculate the number of days rented
    DATE_PART('day', r.return_date - r.rental_date) AS days_rented
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

In [10]:
rsq("""SELECT f.title, f.rental_duration, 
    -- Calculate the number of days rented
    julianday(r.return_date) - julianday(r.rental_date) AS days_rented
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

,title,rental_duration,days_rented
0,ACE GOLDFINGER,3,3.050000
1,ACE GOLDFINGER,3,NaN
2,ACE GOLDFINGER,3,3.050000
3,ACE GOLDFINGER,3,NaN
4,ACE GOLDFINGER,3,1.089583
...,...,...,...
5839,AIRPORT POLLOCK,6,1.062500
5840,AIRPORT POLLOCK,6,8.068056
5841,AIRPORT POLLOCK,6,3.130556
5842,AIRPORT POLLOCK,6,1.062500


In [ ]:
rsq("""SELECT f.title, f.rental_duration,
	-- Calculate the number of days rented
	AGE(r.return_date, r.rental_date) AS days_rented
FROM film AS f
	INNER JOIN inventory AS i ON f.film_id = i.film_id
	INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

In [12]:
rsq("""SELECT f.title, f.rental_duration,
	-- Calculate the number of days rented
	julianday(r.return_date) - julianday(r.rental_date) AS days_rented
FROM film AS f
	INNER JOIN inventory AS i ON f.film_id = i.film_id
	INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

,title,rental_duration,days_rented
0,ACE GOLDFINGER,3,3.050000
1,ACE GOLDFINGER,3,NaN
2,ACE GOLDFINGER,3,3.050000
3,ACE GOLDFINGER,3,NaN
4,ACE GOLDFINGER,3,1.089583
...,...,...,...
5839,AIRPORT POLLOCK,6,1.062500
5840,AIRPORT POLLOCK,6,8.068056
5841,AIRPORT POLLOCK,6,3.130556
5842,AIRPORT POLLOCK,6,1.062500


In [ ]:
rsq("""SELECT INTERVAL '1' day * timestamp '2019-04-10 12:34:56'""")

In [22]:
rsq("""SELECT DATE('2019-04-10 12:34:56', '+1 day')""")


,"DATE('2019-04-10 12:34:56', '+1 day')"
0,2019-04-11


In [ ]:
rsq("""SELECT
	f.title,
 	-- Convert the rental_duration to an interval
    rental_duration * INTERVAL '1' day,
 	-- Calculate the days rented as we did previously
    r.return_date - rental_date AS days_rented
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
-- Filter the query to exclude outstanding rentals
WHERE r.return_date IS NOT NULL
ORDER BY f.title;""")

In [32]:
rsq("""SELECT
    f.title,
    -- Convert the rental_duration to an interval
    rental_duration || ' day',
    -- Calculate the days rented as we did previously
    r.return_date - rental_date AS days_rented
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
-- Filter the query to exclude outstanding rentals
WHERE r.return_date IS NOT NULL
ORDER BY f.title;""")



,title,rental_duration || ' day',days_rented
0,ACE GOLDFINGER,3 day,0
1,ACE GOLDFINGER,3 day,0
2,ACE GOLDFINGER,3 day,0
3,ACE GOLDFINGER,3 day,0
4,ACE GOLDFINGER,3 day,0
...,...,...,...
5720,AIRPORT POLLOCK,6 day,0
5721,AIRPORT POLLOCK,6 day,0
5722,AIRPORT POLLOCK,6 day,0
5723,AIRPORT POLLOCK,6 day,0


In [ ]:
rsq("""SELECT
    f.title,
	r.rental_date,
    f.rental_duration,
    -- Add the rental duration to the rental date
    INTERVAL '1' day * f.rental_duration + r.rental_date AS expected_return_date,
    r.return_date
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;""")

In [35]:
rsq("""SELECT
    f.title,
    r.rental_date,
    f.rental_duration,
    -- Add the rental duration to the rental date
    date(r.rental_date, '+' || f.rental_duration || ' day') AS expected_return_date,
    r.return_date
FROM film AS f
    INNER JOIN inventory AS i ON f.film_id = i.film_id
    INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title; """)


,title,rental_date,rental_duration,expected_return_date,return_date
0,ACE GOLDFINGER,2005-08-01 04:24:47,3,2005-08-04,2005-08-04 05:36:47
1,ACE GOLDFINGER,2006-02-14 15:16:03,3,2006-02-17,None
2,ACE GOLDFINGER,2005-08-01 04:24:47,3,2005-08-04,2005-08-04 05:36:47
3,ACE GOLDFINGER,2006-02-14 15:16:03,3,2006-02-17,None
4,ACE GOLDFINGER,2005-07-07 19:46:51,3,2005-07-10,2005-07-08 21:55:51
...,...,...,...,...,...
5839,AIRPORT POLLOCK,2005-07-29 07:20:14,6,2005-08-04,2005-07-30 08:50:14
5840,AIRPORT POLLOCK,2005-08-17 11:25:25,6,2005-08-23,2005-08-25 13:03:25
5841,AIRPORT POLLOCK,2005-05-31 08:04:17,6,2005-06-06,2005-06-03 11:12:17
5842,AIRPORT POLLOCK,2005-07-29 07:20:14,6,2005-08-04,2005-07-30 08:50:14


In [ ]:
rsq("""-- Select the current timestamp
SELECT now();""")

In [37]:
rsq("""-- Select the current timestamp
SELECT CURRENT_TIMESTAMP;""")


,CURRENT_TIMESTAMP
0,2023-10-20 16:26:23


In [39]:
rsq("""-- Select the current date
SELECT CURRENT_DATE;""")

,CURRENT_DATE
0,2023-10-20


In [38]:
rsq("""SELECT DATE('now');""")

,DATE('now')
0,2023-10-20


In [ ]:
rsq("""--Select the current timestamp without a timezone
SELECT CAST( NOW() AS timestamp )""")

In [41]:
rsq("""--Select the current timestamp without a timezone
SELECT CAST( CURRENT_TIMESTAMP AS timestamp )""")   

,CAST( CURRENT_TIMESTAMP AS timestamp )
0,2023


In [ ]:
rsq("""SELECT 
	-- Select the current date
	NOW(),
    -- CAST the result of the NOW() function to a date
    CAST(NOW() AS date)""")

In [44]:
rsq("""SELECT 
    -- Select the current date
    datetime('now'),
    -- CAST the result of the NOW() function to a date
    CAST(datetime('now') AS date)""")


,datetime('now'),CAST(datetime('now') AS date)
0,2023-10-20 16:33:47,2023


In [ ]:
rsq("""--Select the current timestamp without timezone
SELECT CURRENT_TIMESTAMP::TIMESTAMP AS right_now;""")

In [46]:
rsq("""--Select the current timestamp without timezone
SELECT CURRENT_TIMESTAMP AS right_now;""")


,right_now
0,2023-10-20 16:35:59


In [ ]:
rsq("""SELECT
	CURRENT_TIMESTAMP::timestamp AS right_now,
    CURRENT_TIMESTAMP + INTERVAL '5 days' AS five_days_from_now;""")

In [48]:
rsq("""SELECT
    datetime(CURRENT_TIMESTAMP) AS right_now,
    datetime(CURRENT_TIMESTAMP, '+5 days') AS five_days_from_now;""")


,right_now,five_days_from_now
0,2023-10-20 16:38:05,2023-10-25 16:38:05


In [ ]:
rsq("""SELECT
	CURRENT_TIMESTAMP(0)::timestamp AS right_now,
    interval '5 days' + CURRENT_TIMESTAMP(0) AS five_days_from_now;""")

In [57]:
rsq("""SELECT
    datetime(CURRENT_TIMESTAMP) AS right_now,
    datetime(CURRENT_TIMESTAMP, '+5 days') AS five_days_from_now;""")


,right_now,five_days_from_now
0,2023-10-20 16:48:25,2023-10-25 16:48:25


In [ ]:
rsq("""-- Extract day of week from rental_date
SELECT 
  EXTRACT(dow FROM rental_date) AS dayofweek, 
  -- Count the number of rentals
  COUNT(*) AS rentals 
FROM rental 
GROUP BY 1;""")

In [66]:
rsq("""SELECT 
    strftime('%w', rental_date) AS day_of_week, 
    COUNT(*) AS rentals 
FROM rental 
GROUP BY day_of_week;""")


,day_of_week,rentals
0,0,10
1,1,17
2,2,25
3,3,110
4,4,17
5,5,13
6,6,8


In [69]:
rsq("""SELECT 
    strftime('%Y-%m-%d', rental_date) AS rental_day,
    strftime('%Y-%m', rental_date) AS rental_month,
    strftime('%Y', rental_date) AS rental_year
    -- Count total number of rentals  
FROM rental""")

,rental_day,rental_month,rental_year
0,2005-05-24,2005-05,2005
1,2005-05-24,2005-05,2005
2,2005-05-24,2005-05,2005
3,2005-05-24,2005-05,2005
4,2005-05-24,2005-05,2005
...,...,...,...
195,2005-07-29,2005-07,2005
196,2005-08-01,2005-08,2005
197,2005-05-25,2005-05,2005
198,2005-07-06,2005-07,2005


In [ ]:
rsq("""SELECT 
  DATE_TRUNC('day', rental_date) AS rental_day,
  -- Count total number of rentals 
  COUNT(*) AS rentals  
FROM rental
GROUP BY 1;""")

In [72]:
rsq("""SELECT 
    strftime('%d', rental_date) AS rental_day,
    -- Count total number of rentals 
    COUNT(*) AS rentals  
FROM rental
GROUP BY 1;""")


,rental_day,rentals
0,01,6
1,02,5
2,06,6
3,07,1
4,08,4
5,09,2
6,10,2
7,11,2
8,12,3
9,14,3


In [ ]:
rsq("""SELECT 
  -- Extract the day of week date part from the rental_date
  EXTRACT(DOW FROM rental_date) AS dayofweek,
  AGE(return_date, rental_date) AS rental_days
FROM rental AS r 
WHERE 
  -- Use an INTERVAL for the upper bound of the rental_date 
  rental_date BETWEEN CAST('2005-05-01' AS DATE) 
   AND CAST('2005-05-01' AS DATE) + INTERVAL '90 day';""")

In [91]:
rsq("""SELECT 
  strftime('%w', rental_date) AS dayofweek,
  (julianday(return_date) - julianday(rental_date)) AS rental_days
FROM rental AS r
WHERE
  rental_date BETWEEN date('2005-05-01')
   AND date('2005-05-01', '+90 days');""")

,dayofweek,rental_days
0,2,1.965972
1,2,3.865278
2,2,7.964583
3,2,9.110417
4,2,8.227778
...,...,...
147,4,7.045139
148,5,1.062500
149,3,7.161111
150,3,3.756250


In [ ]:
rsq("""
SELECT  
  c.first_name || ' ' || c.last_name AS customer_name,
  f.title,
  r.rental_date,
  -- Extract the day of week date part from the rental_date
  EXTRACT(dow FROM r.rental_date) AS dayofweek,
  AGE(r.return_date, r.rental_date) AS rental_days,
  -- Use DATE_TRUNC to get days from the AGE function
  CASE 
    WHEN DATE_PART('day', AGE(r.return_date, r.rental_date)) > 
    -- Calculate number of days the rental was overdue
    f.rental_duration THEN 
    DATE_PART('day', AGE(r.return_date, r.rental_date)) - f.rental_duration 
    ELSE 0 
  END AS past_due
FROM 
  rental r 
  JOIN inventory i ON r.inventory_id = i.inventory_id 
  JOIN film f ON i.film_id = f.film_id 
  JOIN customer c ON c.customer_id = r.customer_id 
WHERE  
  -- Use an INTERVAL for the upper bound of the rental_date 
  r.rental_date BETWEEN CAST('2005-05-01' AS DATE)  
  AND CAST('2005-05-01' AS DATE) + INTERVAL '90 day';""")

In [90]:

rsq("""
    SELECT
        c.first_name || ' ' || c.last_name AS customer_name,
        f.title,
        r.rental_date,
        -- Extract the day of week date part from the rental_date
        STRFTIME('%w', r.rental_date) AS dayofweek,
        JULIANDAY(r.return_date) - JULIANDAY(r.rental_date) AS rental_days,
        -- Calculate number of days the rental was overdue
        CASE  
            WHEN JULIANDAY(r.return_date) - JULIANDAY(r.rental_date) > f.rental_duration THEN
                JULIANDAY(r.return_date) - JULIANDAY(r.rental_date) - f.rental_duration
        END AS past_due
    FROM
        rental r
        JOIN inventory i ON r.inventory_id = i.inventory_id
        JOIN film f ON i.film_id = f.film_id
        JOIN customer c ON c.customer_id = r.customer_id
    WHERE
        -- Use an INTERVAL for the upper bound of the rental_date
        r.rental_date BETWEEN DATE('2005-05-01') AND DATE('2005-05-01', '+90 day')
""")

,customer_name,title,rental_date,dayofweek,rental_days,past_due
0,KATHRYN COLEMAN,ACE GOLDFINGER,2005-07-28 05:04:47,4,8.005556,5.005556
1,KATHRYN COLEMAN,ACE GOLDFINGER,2005-07-28 05:04:47,4,8.005556,5.005556
2,KATHRYN COLEMAN,ACE GOLDFINGER,2005-07-28 05:04:47,4,8.005556,5.005556
3,KATHRYN COLEMAN,ACE GOLDFINGER,2005-07-28 05:04:47,4,8.005556,5.005556
4,KATHRYN COLEMAN,ACE GOLDFINGER,2005-07-28 05:04:47,4,8.005556,5.005556
...,...,...,...,...,...,...
728,PHYLLIS FOSTER,AIRPLANE SIERRA,2005-07-11 17:00:04,1,0.844444,NaN
729,PHYLLIS FOSTER,AIRPLANE SIERRA,2005-07-11 17:00:04,1,0.844444,NaN
730,PHYLLIS FOSTER,AIRPLANE SIERRA,2005-07-11 17:00:04,1,0.844444,NaN
731,PHYLLIS FOSTER,AIRPLANE SIERRA,2005-07-11 17:00:04,1,0.844444,NaN
